In [1]:
import pandas as pd

In [5]:
path = 'C:/Users/zolboy/Documents/GitHub/Machine_Learning_OpenClassrooms/Customer Segmentation/data/'
olist_customers_dataset = pd.read_csv(path + 'olist_customers_dataset.csv', sep=',')
olist_geolocation_dataset = pd.read_csv(path + 'olist_geolocation_dataset.csv', sep=',')
olist_order_items_dataset = pd.read_csv(path + 'olist_order_items_dataset.csv', sep=',')
olist_order_payments_dataset = pd.read_csv(path + 'olist_order_payments_dataset.csv', sep=',')
olist_order_reviews_dataset = pd.read_csv(path + 'olist_order_reviews_dataset.csv', sep=',')
olist_orders_dataset = pd.read_csv(path + 'olist_orders_dataset.csv', sep=',')
olist_products_dataset = pd.read_csv(path + 'olist_products_dataset.csv', sep=',')
olist_sellers_dataset = pd.read_csv(path + 'olist_sellers_dataset.csv', sep=',')
product_category_name_translation = pd.read_csv(path + 'product_category_name_translation.csv', sep=',')


All_Datasets = {'olist_customers_dataset' : olist_customers_dataset, 'olist_geolocation_dataset' : olist_geolocation_dataset, 'olist_order_items_dataset' : olist_order_items_dataset, 'olist_order_payments_dataset' : olist_order_payments_dataset, 'olist_order_reviews_dataset' : olist_order_reviews_dataset, 'olist_orders_dataset' : olist_orders_dataset, 'olist_products_dataset' : olist_products_dataset, 'olist_sellers_dataset' : olist_sellers_dataset, 'product_category_name_translation' : product_category_name_translation}


In [13]:
# Befor merge the data we need to see our connection between files csv (databases connection)
for key,value in All_Datasets.items():
    value.info()
    print('------------')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             10001

In [18]:
# Now we need to merge our data

Data1 = pd.merge(olist_customers_dataset, olist_orders_dataset)
Data1 = pd.merge(Data1, olist_orders_dataset)
Data2 = pd.merge(olist_order_payments_dataset, olist_order_items_dataset)
Data2 = pd.merge(Data2, olist_order_reviews_dataset)
Data = pd.merge(Data1, Data2)
Data = pd.merge(Data, olist_products_dataset)
Data = pd.merge(Data, olist_sellers_dataset)
Data.shape

(118315, 39)

In [19]:
# Drop Duplicate Orders

Data.drop_duplicates(subset = "order_id", keep = False, inplace = True)
Data.shape

(85739, 39)

In [20]:
from datetime import datetime
from datetime import date

Data['order_delivered_customer_date'] = pd.to_datetime(Data['order_delivered_customer_date'], format='%Y-%m-%d %H:%M:%S')


In [ ]:
# Do a RFM dataframe (Recency, Frequency, Amount) + Mean Review + Main Categories

Data = Data.dropna(subset=['order_delivered_customer_date'])

Data.drop_duplicates(subset = "order_id", keep = False, inplace = True)

UniqueIndividuals = Data.customer_unique_id.unique()
    

#  Count for each individual, the number of orders, the date of the last order, the total amount spent, and the satisfaction level

print(len(UniqueIndividuals))

List = []

for i in range(len(UniqueIndividuals)):
    Blank = []
    Blank.append(UniqueIndividuals[i])
    Blank.append(Data['order_delivered_customer_date'][Data["customer_unique_id"] == UniqueIndividuals[i]].max())
    Blank.append(len(Data[Data["customer_unique_id"] == UniqueIndividuals[i]]))
    Blank.append(Data['price'][Data["customer_unique_id"] == UniqueIndividuals[i]].sum())
    Blank.append(Data['review_score'][Data["customer_unique_id"] == UniqueIndividuals[i]].mean())
    Blank.append(Data['customer_city'][Data["customer_unique_id"] == UniqueIndividuals[i]].iloc[-1])
    Blank.append(Data['customer_state'][Data["customer_unique_id"] == UniqueIndividuals[i]].iloc[-1])
    Blank.append(Data['payment_type'][Data["customer_unique_id"] == UniqueIndividuals[i]].iloc[-1])
    Blank.append(Data['product_category_name'][Data["customer_unique_id"] == UniqueIndividuals[i]].iloc[-1])
    List.append(Blank)
    if i%1000 == 0:
        print(i)
        
# Create Empty DataFrame with only Individuals and their 'customer_unique_id'

Individuals = pd.DataFrame(List, columns = ['customer_unique_id', 'Recency', 'Frequency', 'Amount', 'Mean_review_score', 'City', 'State', 'Payment_Type', 'Category'])
Individuals.head()        

In [ ]:
# Import Individuals data

import pandas as pd

NewIndividuals = pd.read_csv('/content/IndividualsRFM_Categories.csv', sep=',')
NewIndividuals = NewIndividuals.iloc[:,1::]

NewIndividuals.columns = ['customer_unique_id', 'Frequency', 'Recency', 'Amount', 'Mean_review_score', 'City', 'State', 'Payment_Type', 'Category']


# Change Rencency into numeric (number of days passed since 31/12/2015)

from sklearn.decomposition import KernelPCA
from datetime import datetime
from datetime import date

NewIndividuals['Recency'] = pd.to_datetime(NewIndividuals['Recency'], format='%Y-%m-%d %H:%M:%S')

NewIndividuals['Year'] = NewIndividuals['Recency'].dt.strftime('%Y')
NewIndividuals = NewIndividuals.dropna(subset=['Year'])
NewIndividuals['Year'] = pd.to_numeric(NewIndividuals['Year'])
NewIndividuals['Year'] = NewIndividuals['Year'] - NewIndividuals['Year'].min()

NewIndividuals['Months'] = NewIndividuals['Recency'].dt.strftime('%m')
NewIndividuals = NewIndividuals.dropna(subset=['Months'])
NewIndividuals['Months'] = pd.to_numeric(NewIndividuals['Months'])

NewIndividuals['Days'] = NewIndividuals['Recency'].dt.strftime('%d')
NewIndividuals = NewIndividuals.dropna(subset=['Days'])
NewIndividuals['Days'] = pd.to_numeric(NewIndividuals['Days'])

NewIndividuals['NumRecency'] = 365*NewIndividuals['Year'] + 30*NewIndividuals['Months'] + NewIndividuals['Days']

NewIndividuals['Year'] = NewIndividuals['Recency'].dt.strftime('%Y')
NewIndividuals['Year'] = pd.to_numeric(NewIndividuals['Year'])

NewIndividuals['Months'] = NewIndividuals['Recency'].dt.strftime('%m')
NewIndividuals['Months'] = pd.to_numeric(NewIndividuals['Months'])

NewIndividuals['Days'] = NewIndividuals['Recency'].dt.strftime('%d')
NewIndividuals['Days'] = pd.to_numeric(NewIndividuals['Days'])

NewIndividuals = NewIndividuals[['customer_unique_id', 'Year', 'Months', 'Days', 'Recency', 'Frequency', 'NumRecency', 'Amount', 'Mean_review_score', 'City', 'State', 'Payment_Type', 'Category']]
